In [1]:
import pickle
from sentence_transformers import SentenceTransformer
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

2022-04-21 18:41:12.359016: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu
2022-04-21 18:41:12.359033: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package punkt to /home/anirudh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/anirudh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
with open('../Data/lyrics/merged_lyrics.pkl', 'rb') as file:
    lyrics_dict = pickle.load(file)
with open('../Data/song_spotify_id_mapping.pkl', 'rb') as file:
    song_id_map = pickle.load(file)
lyrics_id_map = {}
for song, id_ in song_id_map.items():
    if song in lyrics_dict:
        lyrics_id_map[id_] = ' '.join(lyrics_dict[song].replace('\r', '\n').split('\n')[1:])

In [3]:
for id_, lyrics in lyrics_id_map.items():
    lyrics = lyrics.lower().replace('-', ' ')
    lyrics = lyrics[:-5]
    ord0 = ord('0')
    for i in range(len(lyrics) - 1, - 1, -1):
        if ord(lyrics[i]) - ord0 > 9 or ord(lyrics[i]) < ord0:
            break
    lyrics = lyrics[:i + 1]
    def add_g(token):
        if token[-1] == "'":
            token = token[:-1] + 'g'
        return token
    lyrics = ' '.join([add_g(token) for token in lyrics.split()])
    tokens = nltk.word_tokenize(lyrics)
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [token for token in tokens if len(token) > 1]
    lyrics = ' '.join(tokens)
    lyrics_id_map[id_] = lyrics

In [13]:
miniLM_model = SentenceTransformer('all-MiniLM-L6-v2')

In [14]:
miniLM_sentence_embeddings = miniLM_model.encode(list(lyrics_id_map.values()))

In [16]:
with open('../Data/embeddings/lyrics_embeddings_minLM.pkl', 'wb') as file:
    keys = list(lyrics_id_map.keys())
    pickle.dump({keys[i]: miniLM_sentence_embeddings[i] for i in range(len(keys))}, file, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
mpnet_model = SentenceTransformer('all-mpnet-base-v2')

OSError: Unable to load weights from pytorch checkpoint file for '/home/anirudh/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/pytorch_model.bin' at '/home/anirudh/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/pytorch_model.bin'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.

In [ ]:
mpnet_sentence_embeddings = mpnet_model.encode(list(lyrics_id_map.values()))

In [ ]:
with open('../Data/embeddings/lyrics_embeddings_mpnet.pkl', 'wb') as file:
    keys = list(lyrics_id_map.keys())
    pickle.dump({keys[i]: mpnet_sentence_embeddings[i] for i in range(len(keys))}, file, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
import gensim
from gensim import corpora, models, similarities
import gensim.downloader as api
word2vec_model300 = api.load('word2vec-google-news-300')
glove_model300 = api.load('glove-wiki-gigaword-300')

In [5]:
texts = [[text for text in lyrics.split()] for lyrics in lyrics_id_map.values()]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [8]:
import numpy as np
word2vec_sentence_embeddings = np.zeros((len(lyrics_id_map.values()), 300), dtype=np.float32)
for i, lyrics in enumerate(lyrics_id_map.values()):
    count = 0
    for token in lyrics.split():
        if token in word2vec_model300:
            count += 1
            word2vec_sentence_embeddings[i] += word2vec_model300[token]
    if count > 0:
        word2vec_sentence_embeddings[i] /= count

In [9]:
with open('../Data/embeddings/lyrics_embeddings_word2_vec.pkl', 'wb') as file:
    keys = list(lyrics_id_map.keys())
    pickle.dump({keys[i]: word2vec_sentence_embeddings[i] for i in range(len(keys))}, file, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
glove_sentence_embeddings = np.zeros((len(lyrics_id_map.values()), 300), dtype=np.float32)
for i, lyrics in enumerate(lyrics_id_map.values()):
    count = 0
    for token in lyrics.split():
        if token in glove_model300:
            count += 1
            glove_sentence_embeddings[i] += glove_model300[token]
    if count > 0:
        glove_sentence_embeddings[i] /= count

In [11]:
with open('../Data/embeddings/lyrics_embeddings_glove.pkl', 'wb') as file:
    keys = list(lyrics_id_map.keys())
    pickle.dump({keys[i]: glove_sentence_embeddings[i] for i in range(len(keys))}, file, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
from gensim.models import Word2Vec
model = Word2Vec(texts, min_count = 1, vector_size=300)
word2vec_trained_embeddings = np.zeros((len(lyrics_id_map.values()), 300), dtype=np.float32)
for i, lyrics in enumerate(lyrics_id_map.values()):
    for token in lyrics.split():
        word2vec_trained_embeddings[i] += model.wv[token]
    word2vec_trained_embeddings[i] /= len(lyrics.split())

/tmp/ipykernel_5679/2804340455.py:7: RuntimeWarning: invalid value encountered in true_divide
  word2vec_trained_embeddings[i] /= len(lyrics.split())


In [13]:
with open('../Data/embeddings/lyrics_embeddings_word2vec_trained.pkl', 'wb') as file:
    keys = list(lyrics_id_map.keys())
    pickle.dump({keys[i]: word2vec_trained_embeddings[i] for i in range(len(keys))}, file, protocol=pickle.HIGHEST_PROTOCOL)